<a href="https://colab.research.google.com/github/GalindoD/Backtesting_crypto/blob/main/Backtester_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download Requirements

In [1]:
pip install --upgrade yfinance

In [2]:
pip install backtrader yfinance matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 17.7 MB/s eta 0:00:00


In [3]:

import pandas as pd
import numpy as np
import plotly.express as px
import backtrader as bt
import yfinance as yf
import matplotlib.pyplot as plt
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
!pip install conda-package-handling
!wget https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
!cph x ta-lib-0.5.1-py311h9ecbd09_0.conda
!mv ./ta-lib-0.5.1-py311h9ecbd09_0/lib/python3.11/site-packages/talib /usr/local/lib/python3.11/dist-packages/
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4087    0  4087    0     0  13098      0 --:--:-- --:--:-- --:--:-- 13141
100  517k  100  517k    0     0   707k      0 --:--:-- --:--:-- --:--:--  707k
--2025-08-01 00:01:49--  https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
Resolving anaconda.org (anaconda.org)... 104.19.145.37, 104.19.144.37, 2606:4700::6813:9125, ...
Connecting to anaconda.org (anaconda.org)|104.19.145.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5f738b4b78d4370a69f82984/673b62920c609ff6945bb24d?response-content-disposition=attachment%3B%20filename%3D%22ta-lib-0.5.1-py311h9ecbd09_0.conda%22%3B%20filename%2A%3DUTF-8%27%27ta-lib-0.5.1-py311h9ecbd09_0.conda&response-content-type=application%2Foctet-stream&X-Amz-Algorithm=

#Functions for Data gathering and Computing Buy Signals

In [4]:
def gatherdata(ticker, periodicity):
  data = yf.download(
      tickers = ticker,
      #start="2015-03-31",
      #end="2025-05-08",
      period="max",
      interval=periodicity,
      ignore_tz=True,
      auto_adjust=True)
  return data

def createdf(data, fastma, slowma, fastmacd, slowmacd, signal):
  data_df = data.copy()
  data_df.columns = ["close", "high", "low", "open", "volume"]
  data_df.drop(columns=["volume", "high", "low", "open"], inplace=True)

  data_df["R"] = data_df.close.pct_change().fillna(0)

  #Calculating values of Indicators
  data_df["slow_ma"] = talib.SMA(data_df.close, slowma)
  data_df["fast_ma"] = talib.SMA(data_df.close, fastma)
  data_df["macd"], data_df["signal"], data_df["histogram"] = talib.MACD(data_df.close, fastperiod=fastmacd, slowperiod=slowmacd, signalperiod=signal)

  #Delete First NA rows, resultring form lack of previous data
  data_df.dropna(inplace=True)
  return data_df

In [5]:

def signalcompute(data_df):
  data_df = data_df.assign(
      macdbuy = lambda x: np.where( x.macd > x.signal ,1,0)
      )
  data_df = data_df.assign(
      fastmabuy = lambda x: np.where( x.close > x.fast_ma ,1,0)
      )
  data_df = data_df.assign(
      slowmabuy = lambda x: np.where( x.close > x.slow_ma ,1,0)
      )
  return data_df

In [6]:
def compute_strategy(data_df, strategy, liquid_return):
  data_df["buy"] = 0
  match strategy:
    case 1:
      #Strategy 1: MACD and MA, buy both signals 1 and sell both signals 0
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"]  == 1):
          data_df.loc[i, "buy"] = 1
        elif ((data_df.loc[i, "macdbuy"] == 1 or data_df.loc[i, "fastmabuy"] == 1) and data_df.loc[i - pd.Timedelta(days=days), "buy"] == 1):
          data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0
    case 2:
      #Strategy 2: MACD and fastMA, buy both signals 1 and sell when either signal is 0
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"] == 1):
            data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0
    case 3:
      #Strategy 3: Only MACD
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 ):
            data_df.loc[i, "buy"] = 1
        else:
            data_df.loc[i, "buy"] = 0
    case 4:
      #Strategy 4: MACD and SlowMA and FastMA
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"] == 1 and data_df.loc[i, "slowmabuy"] == 1):
          data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0

  #Shift
  data_df["buyshift"] = data_df["buy"].shift(1, fill_value=0)

  #Buying according to strategy
  data_df["R_strategy"] = data_df.R * data_df.buyshift

  #Assuming 5% return on liquid
  data_df['R_strategy'] = data_df['R_strategy'].replace([0], liquid_return)

  #Adding transaction costs to Strategy
  #for i, row in data_df.iterrows():
  #    try:
  #      if data_df.loc[i, "buy"] != data_df.loc[i + pd.Timedelta(days=days), "buy"]:
  #        data_df.loc[i + pd.Timedelta(days=days), "R_strategy"] = data_df.loc[i + pd.Timedelta(days=days), "R_strategy"] - trsncost
  #     else:
  #        continue
  #    except:
  #      print("End")
  data_df['R_strategy'] = [data_df.loc[ei, 'R_strategy'] - trsncost if data_df.loc[ei, 'buy'] != data_df.loc[ei, 'buyshift'] else data_df.loc[ei, 'R_strategy'] for ei in data_df.index]

  return data_df

#Main Cell for declaring variables and strategies

In [7]:
#Main Cell


#Variables
ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"
#ticker = "SOL-USD"
#ticker = "SUI20947-USD"

trsncost = 0.002

periodicity = "5d"
#periodicity = "1wk"
#periodicity = "1d"

if periodicity == "5d":
  liquid_return = 0.0 # 0.000484
elif periodicity == "1wk":
  liquid_return = 0.000958
elif periodicity == "1d":
  liquid_return = 0.000136

#Strategy 1: MACD and FastMA, buy both signals 1 and sell both signals 0. | 5d: 0.658x | 1wk: 0.5089|
#Strategy 2: MACD and FastMA, buy both signals 1 and sell when either signal is 0. | 5d: 0.8033x | 1wk: 0.5063|
#Strategy 3: Only MACD. | 5d: 0.8409x | 1wk:  0.5176|
#Strategy 4: MACD and 40SlowMA and FastMA. | 5d: 0.9272 | 1wk: 0.5263 |
strategy = 4
slowma = 40
fastma = 10

#---Rule1: fastmacd = 8 slowmacd = 17 signal = 9 --- OPtimized: Slowmacd = 19
fastmacd = 8
slowmacd = 19
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

#Create dataframe
data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

#Compute the signal
data_df = signalcompute(data_df)

#Choose and compute strategy
data_df = compute_strategy(data_df, strategy, liquid_return)

[*********************100%***********************]  1 of 1 completed


In [8]:
data_df.tail(5)

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,buyshift,R_strategy
Date,,,,,,,,,,,,,
2025-07-11,117516.992188,0.075847,96915.688477,107618.521875,5132.064271,4422.430556,709.633715,1,1,1,1,0,-0.002000
2025-07-16,118738.507812,0.010394,97505.052930,108592.908594,5866.085765,4711.161597,1154.924167,1,1,1,1,1,0.010394
2025-07-21,117439.539062,-0.010940,98018.869531,109771.652344,6090.780502,4987.085378,1103.695124,1,1,1,1,1,-0.010940
2025-07-26,117947.367188,0.004324,98544.486133,111127.354688,6174.784022,5224.625107,950.158915,1,1,1,1,1,0.004324
2025-07-31,115738.953125,-0.018724,99075.749023,111832.587500,5826.451789,5344.990444,481.461346,1,1,1,1,1,-0.018724


#Plot last N values

In [13]:
last_200 = data_df.tail(100)
colors=['red' if val == 0 else 'green' for val in last_200['buy']]
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: 2 rows, 1 column, shared x-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1,
                    #subplot_titles=("Price and Moving Averages", "MACD")
                    )

# Row 1: Price and Moving Averages
fig.add_trace(go.Scatter(x=last_200.index, y=last_200['close'],
                         mode='markers+lines', name='Close', marker={"color":colors, "size":5}),
              row=1, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['slow_ma'],
                         mode='lines', name='SlowMA'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['fast_ma'],
                         mode='lines', name='FastMA'),
              row=1, col=1)

# Row 2: MACD and Signal Line
fig.add_trace(go.Scatter(x=last_200.index, y=last_200['macd'],
                         mode='lines', name='MACD'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['signal'],
                         mode='lines', name='Signal'),
              row=2, col=1)


# Horizontal line at y=0 on MACD plot
fig.add_shape(type="line",
              x0=last_200.index.min(), x1=last_200.index.max(),
              y0=0, y1=0,
              line=dict(color="gray", width=2),
              row=2, col=1)


# Layout and labels
fig.update_layout(height=600, width=900, showlegend=True,
                  title_text=f"Price, MA, MACD: {ticker}.")

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="MACD", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

fig.show()

#Plot returns

In [ ]:
#Total Return with transaction costs
returns = 100 * (1+data_df[["R","R_strategy"]]).prod()-1
print("With transaction cost:")
print("Return of Holding: ", returns.iloc[0])
print("Return of Strategy: ", returns.iloc[1])
print("The return of this strategy is ",round((returns.iloc[1]/returns.iloc[0]).item(), 4), "x of holding. Ticker: ", ticker)

With transaction cost:
Return of Holding:  47467.048445851455
Return of Strategy:  32907.00481809777
The return of this strategy is  0.6933 x of holding. Ticker:  BTC-USD


In [ ]:
px.line(100 * (1 + data_df[["R","R_strategy"]]).cumprod(), title=f"Total Return: {ticker} {periodicity} -> FMCD: {fastmacd} SMCD: {slowmacd} SIG: {signal}.")

#Loop cell for finding best parameters

In [ ]:
#Loop Cell la buena


#Variables
ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"

trsncost = 0.002

periodicity = "5d"
#periodicity = "1wk"
#periodicity = "1d"

if periodicity == "5d":
  days =  5
  liquid_return = 0.000484
elif periodicity == "1wk":
  days = 7
  liquid_return = 0.000958
elif periodicity == "1d":
  days = 1
  liquid_return = 0.000136

strategy = 4
slowma = 40
fastma = 10

#---Rule1: fastmacd = 8 slowmacd = 17 signal = 9
fastmacd = 8
slowmacd = 17
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

row_name = "slowmacd_" + str(slowmacd)
column_name = "fastmacd_" + str(fastmacd)

returns_df = pd.DataFrame({column_name: [0]}, index=[row_name])

for slowmacd in range(17, 29):
  row_name = "slowmacd_" + str(slowmacd)
  #returns_df.loc[row_name, column_name] = pd.DataFrame({'Values': [row_name]})

  for fastmacd in range(7, 15):
    #Create dataframe
    data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

    #Compute the signal
    data_df = signalcompute(data_df)

    #Choose and compute strategy
    data_df = compute_strategy(data_df, strategy, liquid_return)

    #Total Return with transaction costs
    retvals = (100 * (1+data_df[["R_strategy"]]).prod()-1).item()
    column_name = "fastmacd_" + str(fastmacd)
    returns_df.loc[row_name, column_name] =  retvals

[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-13-3382554565.py:59: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '40031.17357670356' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [ ]:
returns_df

,fastmacd_8,fastmacd_7,fastmacd_9,fastmacd_10,fastmacd_11,fastmacd_12,fastmacd_13,fastmacd_14
slowmacd_17,40031.173577,25189.179839,41063.102665,35631.774349,33247.014105,30435.610242,30435.610242,32390.417080
slowmacd_18,39535.373991,29510.692592,35631.774349,35135.290044,30435.610242,30435.610242,32390.417080,35676.596039
slowmacd_19,41063.102665,40031.173577,35631.774349,35558.352670,30435.610242,30873.936131,35390.418370,36380.017345
slowmacd_20,39097.766485,39997.844464,38301.016337,31239.243351,30435.610242,35390.418370,35676.596039,35669.076854
slowmacd_21,35631.774349,39535.373991,34151.467885,32164.216305,30873.936131,35390.418370,35676.596039,34240.479614
slowmacd_22,31393.397235,38347.523871,34151.467885,32164.216305,35390.418370,35676.596039,34691.717039,33561.123786
slowmacd_23,34151.467885,39097.766485,34053.941763,32164.216305,35390.418370,35676.596039,33578.425937,39794.046076
slowmacd_24,34151.467885,31393.397235,35156.433519,33275.353478,35390.418370,34691.717039,39024.615056,38458.751271
slowmacd_25,34151.467885,31393.397235,35156.433519,37400.426051,37702.856794,33578.425937,39024.615056,38458.751271
slowmacd_26,34151.467885,31771.405162,38412.615416,37400.426051,36966.066005,41241.022017,40643.020764,37276.816780


In [ ]:
fig = px.imshow(returns_df, text_auto=True, aspect="auto")
fig.show()

#Another cell for testing and computing Misc

In [ ]:
#Main Cell


#Variables
#ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"
#ticker = "SOL-USD"
ticker = "SUI20947-USD"

trsncost = 0.002

#periodicity = "5d"
#periodicity = "1wk"
periodicity = "1d"

if periodicity == "5d":
  days =  5
  liquid_return = 0.000684
elif periodicity == "1wk":
  days = 7
  liquid_return = 0.000958
elif periodicity == "1d":
  days = 1
  liquid_return = 0.000136

#Strategy 1: MACD and FastMA, buy both signals 1 and sell both signals 0. | 5d: 0.658x | 1wk: 0.5089|
#Strategy 2: MACD and FastMA, buy both signals 1 and sell when either signal is 0. | 5d: 0.8033x | 1wk: 0.5063|
#Strategy 3: Only MACD. | 5d: 0.8409x | 1wk:  0.5176|
#Strategy 4: MACD and 40SlowMA and FastMA. | 5d: 0.9272 | 1wk: 0.5263 |
strategy = 4
slowma = 40
fastma = 10

#---Rule1: fastmacd = 8 slowmacd = 17 signal = 9
fastmacd = 8
slowmacd = 19
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

#Create dataframe
data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

#Compute the signal
data_df = signalcompute(data_df)

#Choose and compute strategy
data_df = compute_strategy(data_df, strategy, liquid_return)




[*********************100%***********************]  1 of 1 completed


In [ ]:
#Total Return with transaction costs
returns = 100 * (1+data_df[["R","R_strategy"]]).prod()-1
print("With transaction cost:")
print("Return of Holding: ", returns.iloc[0])
print("Return of Strategy: ", returns.iloc[1])
print("The return of this strategy is ",round((returns.iloc[1]/returns.iloc[0]).item(), 4), "x of holding. Ticker: ", ticker)

With transaction cost:
Return of Holding:  459.27025593088416
Return of Strategy:  183.29168845828673
The return of this strategy is  0.3991 x of holding. Ticker:  SUI20947-USD


In [ ]:
px.line(100 * (1 + data_df[["R","R_strategy"]]).cumprod(), title=f"Total Return: {ticker} {periodicity} -> FMCD: {fastmacd} SMCD: {slowmacd} SIG: {signal}.")

In [ ]:
data_df.tail(20)

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,R_strategy
Date,,,,,,,,,,,,
2025-01-24,253.361221,0.011073,177.602727,211.329198,14.352357,11.999282,2.353075,1,1,1,0,0.000684
2025-01-29,227.944931,-0.100316,179.753226,212.741229,13.876844,12.374794,1.502049,1,1,1,1,-0.102316
2025-02-03,216.149582,-0.051746,181.180155,211.990421,11.793956,12.258627,-0.464671,0,1,1,1,-0.051746
2025-02-08,199.599899,-0.076566,181.851213,212.529593,8.051108,11.417123,-3.366015,0,0,1,0,-0.001316
2025-02-13,194.460953,-0.025746,182.090585,212.227780,4.624107,10.058520,-5.434413,0,0,1,0,0.000684
2025-02-18,169.084290,-0.130497,182.501736,210.033405,-0.979144,7.850987,-8.830131,0,0,0,0,0.000684
2025-02-23,168.038116,-0.006187,183.086887,205.174924,-5.007526,5.279284,-10.286810,0,0,0,0,0.000684
2025-02-28,148.030014,-0.119069,183.126281,201.475397,-10.161550,2.191117,-12.352668,0,0,0,0,0.000684
2025-03-05,146.267303,-0.011908,183.238001,197.352267,-13.758997,-0.998905,-12.760092,0,0,0,0,0.000684


In [ ]:
last_200 = data_df.tail(200)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: 2 rows, 1 column, shared x-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1,
                    #subplot_titles=("Price and Moving Averages", "MACD")
                    )

# Row 1: Price and Moving Averages
fig.add_trace(go.Scatter(x=last_200.index, y=last_200['close'],
                         mode='lines', name='Close'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['slow_ma'],
                         mode='lines', name='SlowMA'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['fast_ma'],
                         mode='lines', name='FastMA'),
              row=1, col=1)

# Row 2: MACD and Signal Line
fig.add_trace(go.Scatter(x=last_200.index, y=last_200['macd'],
                         mode='lines', name='MACD'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=last_200.index, y=last_200['signal'],
                         mode='lines', name='Signal'),
              row=2, col=1)

# Horizontal line at y=0 on MACD plot
fig.add_shape(type="line",
              x0=last_200.index.min(), x1=last_200.index.max(),
              y0=0, y1=0,
              line=dict(color="gray", width=2),
              row=2, col=1)

# Layout and labels
fig.update_layout(height=600, width=900, showlegend=True,
                  title_text=f"Price, MA, MACD: {ticker}.")

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="MACD", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

fig.show()

#Test code

In [ ]:
#Main Cell


#Variables
#ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"
#ticker = "SOL-USD"
ticker = "SUI20947-USD"

trsncost = 0.002

periodicity = "5d"
#periodicity = "1wk"
#periodicity = "1d"

if periodicity == "5d":
  days =  5
  liquid_return = 0.0
elif periodicity == "1wk":
  days = 7
  liquid_return = 0.000958
elif periodicity == "1d":
  days = 1
  liquid_return = 0.000136

#Strategy 1: MACD and FastMA, buy both signals 1 and sell both signals 0. | 5d: 0.658x | 1wk: 0.5089|
#Strategy 2: MACD and FastMA, buy both signals 1 and sell when either signal is 0. | 5d: 0.8033x | 1wk: 0.5063|
#Strategy 3: Only MACD. | 5d: 0.8409x | 1wk:  0.5176|
#Strategy 4: MACD and 40SlowMA and FastMA. | 5d: 0.9272 | 1wk: 0.5263 |
strategy = 4
slowma = 40
fastma = 10

#---Rule1: fastmacd = 8 slowmacd = 17 signal = 9
fastmacd = 8
slowmacd = 19
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

#Create dataframe
data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

#Compute the signal
data_df = signalcompute(data_df)

#Choose and compute strategy
data_df = compute_strategy(data_df, strategy, liquid_return)

[*********************100%***********************]  1 of 1 completed


In [ ]:
data_df

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,buyshift,R_strategy
Date,,,,,,,,,,,,,
2023-11-14,0.567980,0.013070,0.662398,0.464913,-0.024688,-0.070202,0.045514,1,1,0,0,0,0.000
2023-11-19,0.564647,-0.005868,0.641435,0.474323,-0.014157,-0.058993,0.044836,1,1,0,0,0,0.000
2023-11-24,0.627237,0.110848,0.628580,0.493573,0.001179,-0.046958,0.048137,1,1,0,0,0,0.000
2023-11-29,0.601067,-0.041723,0.616096,0.512135,0.008690,-0.035829,0.044518,1,1,0,0,0,0.000
2023-12-04,0.628795,0.046131,0.603263,0.533987,0.017143,-0.025234,0.042377,1,1,1,1,0,-0.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-06,3.198134,-0.043170,3.439490,3.484613,0.168227,0.112933,0.055294,1,0,0,0,0,0.000
2025-06-11,3.397876,0.062456,3.438473,3.557712,0.151015,0.120549,0.030466,1,0,0,0,0,0.000
2025-06-16,2.994464,-0.118725,3.427178,3.501440,0.086306,0.113701,-0.027395,0,0,0,0,0,0.000


In [ ]:
BTCdf = data_df.copy()

In [ ]:
BTCdf.rename(columns={"buy": "BTCbuy"}, inplace=True)

In [ ]:
BTCdf

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,BTCbuy,buyshift,R_strategy
Date,,,,,,,,,,,,,
2015-03-31,244.223999,-0.017334,313.141575,260.901698,-11.506411,-16.736175,5.229764,1,0,0,0,0,0.000000
2015-04-05,260.597992,0.067045,308.223174,261.229396,-10.140698,-15.417080,5.276381,1,0,0,0,0,0.000000
2015-04-10,236.072006,-0.094114,304.071174,260.808296,-11.956971,-14.725058,2.768087,1,0,0,0,0,0.000000
2015-04-15,223.832993,-0.051844,299.678999,259.318095,-14.458531,-14.671753,0.213222,1,0,0,0,0,0.000000
2015-04-20,224.626007,0.003543,295.917850,255.760497,-15.791398,-14.895682,-0.895717,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-06,104390.343750,-0.011943,95158.359375,101785.353125,5394.824241,3664.022225,1730.802016,1,1,1,1,1,-0.011943
2025-06-11,108686.625000,0.041156,95431.092969,103309.826563,5554.292386,4042.076257,1512.216129,1,1,1,1,1,0.041156
2025-06-16,106796.757812,-0.017388,95709.700195,104614.017969,5311.507600,4295.962526,1015.545074,1,1,1,1,1,-0.017388


In [ ]:
SUIdf = data_df.copy()

In [ ]:
concat_df = pd.concat([SUIdf, BTCdf["BTCbuy"]], axis=1)
print(concat_df)

               close         R   slow_ma   fast_ma      macd    signal  \
Date                                                                     
2015-03-31       NaN       NaN       NaN       NaN       NaN       NaN   
2015-04-05       NaN       NaN       NaN       NaN       NaN       NaN   
2015-04-10       NaN       NaN       NaN       NaN       NaN       NaN   
2015-04-15       NaN       NaN       NaN       NaN       NaN       NaN   
2015-04-20       NaN       NaN       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-06-06  3.198134 -0.043170  3.439490  3.484613  0.168227  0.112933   
2025-06-11  3.397876  0.062456  3.438473  3.557712  0.151015  0.120549   
2025-06-16  2.994464 -0.118725  3.427178  3.501440  0.086306  0.113701   
2025-06-21  2.557149 -0.146041  3.398533  3.414422 -0.014359  0.088089   
2025-06-26  2.771148  0.083687  3.362462  3.358547 -0.058349  0.058801   

            histogram  macdbuy  fastm

In [ ]:
concat_df

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,buyshift,R_strategy,BTCbuy
Date,,,,,,,,,,,,,,
2015-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2015-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2015-04-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2015-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2015-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-06,3.198134,-0.043170,3.439490,3.484613,0.168227,0.112933,0.055294,1.0,0.0,0.0,0.0,0.0,0.0,1
2025-06-11,3.397876,0.062456,3.438473,3.557712,0.151015,0.120549,0.030466,1.0,0.0,0.0,0.0,0.0,0.0,1
2025-06-16,2.994464,-0.118725,3.427178,3.501440,0.086306,0.113701,-0.027395,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
#Buying according to strategy
concat_df["R_str2"] = concat_df.R * concat_df.BTCbuy

#Assuming 5% return on liquid
concat_df['R_str2'] = concat_df['R_str2'].replace([0], liquid_return)

In [ ]:
concat_df

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,buyshift,R_strategy,BTCbuy,R_str2
Date,,,,,,,,,,,,,,,
2015-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2015-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2015-04-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2015-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2015-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-06,3.198134,-0.043170,3.439490,3.484613,0.168227,0.112933,0.055294,1.0,0.0,0.0,0.0,0.0,0.0,1,-0.043170
2025-06-11,3.397876,0.062456,3.438473,3.557712,0.151015,0.120549,0.030466,1.0,0.0,0.0,0.0,0.0,0.0,1,0.062456
2025-06-16,2.994464,-0.118725,3.427178,3.501440,0.086306,0.113701,-0.027395,0.0,0.0,0.0,0.0,0.0,0.0,1,-0.118725


In [ ]:
px.line(100 * (1 + concat_df[["R","R_strategy","R_str2"]]).cumprod(), title=f"Total Return: {ticker} {periodicity} -> FMCD: {fastmacd} SMCD: {slowmacd} SIG: {signal}.")

#Another cel for testing another way to iterate dataframes

In [ ]:
def compute_strategy(data_df, strategy, days, liquid_return):
  data_df["buy"] = 0
  match strategy:
    case 1:
      #Strategy 1: MACD and MA, buy both signals 1 and sell both signals 0
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"]  == 1):
          data_df.loc[i, "buy"] = 1
        elif ((data_df.loc[i, "macdbuy"] == 1 or data_df.loc[i, "fastmabuy"] == 1) and data_df.loc[i - pd.Timedelta(days=days), "buy"] == 1):
          data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0
    case 2:
      #Strategy 2: MACD and fastMA, buy both signals 1 and sell when either signal is 0
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"] == 1):
            data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0
    case 3:
      #Strategy 3: Only MACD
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 ):
            data_df.loc[i, "buy"] = 1
        else:
            data_df.loc[i, "buy"] = 0
    case 4:
      #Strategy 4: MACD and SlowMA and FastMA
      for i, row in data_df.iterrows():
        if ( data_df.loc[i, "macdbuy"] == 1 and data_df.loc[i, "fastmabuy"] == 1 and data_df.loc[i, "slowmabuy"] == 1):
          data_df.loc[i, "buy"] = 1
        else:
          data_df.loc[i, "buy"] = 0

  #Shift
  data_df["buyshift"] = data_df["buy"].shift(1, fill_value=0)

  #Buying according to strategy
  data_df["R_strategy"] = data_df.R * data_df.buyshift

  #Assuming 5% return on liquid
  data_df['R_strategy'] = data_df['R_strategy'].replace([0], liquid_return)

  #Adding transaction costs to Strategy
  #for i, row in data_df.iterrows():
  #    try:
  #      if data_df.loc[i, "buy"] != data_df.loc[i + pd.Timedelta(days=days), "buy"]:
  #        data_df.loc[i + pd.Timedelta(days=days), "R_strategy"] = data_df.loc[i + pd.Timedelta(days=days), "R_strategy"] - trsncost
  #     else:
  #        continue
  #    except:
  #      print("End")
  data_df['R_strategy'] = [data_df.loc[ei, 'R_strategy'] - trsncost if data_df.loc[ei, 'buy'] != data_df.loc[ei, 'buyshift'] else data_df.loc[ei, 'R_strategy'] for ei in data_df.index]


  return data_df

In [ ]:
#Main Cell


#Variables
ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"
#ticker = "SOL-USD"

trsncost = 0.002

periodicity = "5d"
#periodicity = "1wk"
#periodicity = "1d"

if periodicity == "5d":
  days =  5
  liquid_return = 0.000484
elif periodicity == "1wk":
  days = 7
  liquid_return = 0.000958
elif periodicity == "1d":
  days = 1
  liquid_return = 0.000136

#Strategy 1: MACD and FastMA, buy both signals 1 and sell both signals 0. | 5d: 0.658x | 1wk: 0.5089|
#Strategy 2: MACD and FastMA, buy both signals 1 and sell when either signal is 0. | 5d: 0.8033x | 1wk: 0.5063|
#Strategy 3: Only MACD. | 5d: 0.8409x | 1wk:  0.5176|
#Strategy 4: MACD and 40SlowMA and FastMA. | 5d: 0.9272 | 1wk: 0.5263 |
strategy = 4
slowma = 40
fastma = 10

#---Rule1: fastmacd = 8 slowmacd = 17 signal = 9 --- OPtimized: Slowmacd = 19
fastmacd = 8
slowmacd = 19
signal = 9

#Gather the data
data = gatherdata(ticker, periodicity)

#Create dataframe
data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

#Compute the signal
data_df = signalcompute(data_df)

#Choose and compute strategy
data_df = compute_strategy(data_df, strategy, days, liquid_return)



[*********************100%***********************]  1 of 1 completed


In [ ]:
data_df

,close,R,slow_ma,fast_ma,macd,signal,histogram,macdbuy,fastmabuy,slowmabuy,buy,buyshift,R_strategy
Date,,,,,,,,,,,,,
2015-03-31,244.223999,-0.017334,313.141575,260.901698,-11.506411,-16.736175,5.229764,1,0,0,0,0,0.000484
2015-04-05,260.597992,0.067045,308.223174,261.229396,-10.140698,-15.417080,5.276381,1,0,0,0,0,0.000484
2015-04-10,236.072006,-0.094114,304.071174,260.808296,-11.956971,-14.725058,2.768087,1,0,0,0,0,0.000484
2015-04-15,223.832993,-0.051844,299.678999,259.318095,-14.458531,-14.671753,0.213222,1,0,0,0,0,0.000484
2015-04-20,224.626007,0.003543,295.917850,255.760497,-15.791398,-14.895682,-0.895717,0,0,0,0,0,0.000484
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-17,84895.750000,-0.004589,87820.855664,83831.689844,-2518.174974,-1803.708921,-714.466053,0,1,0,0,0,0.000484
2025-04-22,93441.890625,0.100666,88605.216016,84569.311719,-1182.317308,-1679.430599,497.113291,1,1,1,1,0,-0.001516
2025-04-27,93754.843750,0.003349,89434.534570,85329.336719,-182.693398,-1380.083158,1197.389761,1,1,1,1,1,0.003349


In [ ]:
#Total Return with transaction costs
returns = 100 * (1+data_df[["R","R_strategy"]]).prod()-1
print("With transaction cost:")
print("Return of Holding: ", returns.iloc[0])
print("Return of Strategy: ", returns.iloc[1])
print("The return of this strategy is ",round((returns.iloc[1]/returns.iloc[0]).item(), 4), "x of holding. Ticker: ", ticker)

With transaction cost:
Return of Holding:  40865.57201500536
Return of Strategy:  41377.33610548968
The return of this strategy is  1.0125 x of holding. Ticker:  BTC-USD
